In [ ]:
from typing import Annotated, Sequence, TypedDict, Literal
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START


In [27]:
class AgentState(TypedDict, total=False):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [28]:
def handle_query(state: AgentState) -> str:
    return "this is a tool node"

def tools_node(state: AgentState) -> str:
    return "this is a tool node"

def get_new_query(state: AgentState) -> str:
    return "this is a tool node"

def escalate_to_gmail(state: AgentState) -> str:
    return "this is a tool node"

def end_state(state: AgentState) -> str:
    return "this is a tool node"

In [29]:
def should_continue(state: AgentState) -> Literal["end", "continue"]:
    return "continue"

def ask_another_query(state: AgentState) -> Literal["ask", "no"]:
    return "ask"

def ask_was_helpful(state: AgentState) -> Literal["end", "continue"]:
    return "continue"

def ask_escalate(state: AgentState) -> Literal["end", "continue"]:
    return "continue"

In [30]:
graph = StateGraph(AgentState)

In [31]:
graph.add_node("handle_query", handle_query)
graph.add_node("tool_execution", tools_node)
graph.add_node("ask_another", ask_another_query)
graph.add_node("get_query", get_new_query)
graph.add_node("ask_was_helpful", ask_was_helpful)
graph.add_node("ask_escalate", ask_escalate)
graph.add_node("escalate", escalate_to_gmail)
graph.add_node("end", end_state)

In [32]:
graph.add_node("router0", lambda state: state)
graph.add_node("router1", lambda state: state)
graph.add_node("router2", lambda state: state)

In [33]:
graph.set_entry_point("handle_query")

graph.add_conditional_edges(
    "handle_query",
    should_continue,
    {
        "continue": "tool_execution",
        "end": "router0",
    }
)

graph.add_edge("tool_execution", "router0")

graph.add_conditional_edges(
    "router0",
    ask_another_query,
    {
        "ask": "get_query",
        "no": "router1",
    }
)

graph.add_edge("get_query", "handle_query")

graph.add_conditional_edges(
    "router1",
    ask_was_helpful,
    {
        "end": "end",
        "continue": "router2",
    }
)

graph.add_conditional_edges(
    "router2",
    ask_escalate,
    {
        "end": "end",
        "continue": "escalate",
    }
)

graph.add_edge("escalate", "end")
graph.add_edge("end", END)

app = graph.compile()

In [40]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles


try:
    display(Image(app.get_graph().draw_png()))
except ImportError:
    print(
        "You likely need to install dependencies for pygraphviz, see more here https://github.com/pygraphviz/pygraphviz/blob/main/INSTALL.txt"
    )

You likely need to install dependencies for pygraphviz, see more here https://github.com/pygraphviz/pygraphviz/blob/main/INSTALL.txt
